In [1]:
import numpy as np
import pandas as pd

In [2]:
#importing data
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

C:\Users\adiru\AppData\Local\Temp\ipykernel_1924\3884128499.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('books.csv')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
#checking rows and column count of each data set
print(books.shape)
print(ratings.shape)
print(users.shape)

In [ ]:
#cheking missing values
books.isnull().sum()

In [ ]:
 users.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
users.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

## Popularity Based Recommender System

In [ ]:
 #merging ratings with books
ratings_with_books = ratings.merge(books, on='ISBN')

In [ ]:
 ratings_with_books

In [ ]:
#grouping according to the book-title for it's rating and finding it's sum
num_rating_df = ratings_with_books.groupby('Book-Title').count()['Book-Rating'].reset_index()

In [ ]:
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

In [ ]:
# Identify non-numeric 'Book-Rating' values
non_numeric_ratings = ratings_with_books[pd.to_numeric(ratings_with_books['Book-Rating'], errors='coerce').isna()]
# Now, clean the 'Book-Rating' column by converting it to numeric and dropping NaN values
ratings_with_books['Book-Rating'] = pd.to_numeric(ratings_with_books['Book-Rating'], errors='coerce')
ratings_with_books = ratings_with_books.dropna(subset=['Book-Rating'])
# Group by 'Book-Title' and calculate the mean rating
avg_rating_df = ratings_with_books.groupby('Book-Title').mean(numeric_only=True)['Book-Rating'].reset_index()
# Rename the column for clarity
avg_rating_df.rename(columns={'Book-Rating': 'avg_ratings'}, inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

In [ ]:
popular_df = popular_df[popular_df['num_ratings'] >= 250].sort_values('avg_ratings', ascending=False).head(50)

In [ ]:
#Top 50 books according to ratings/popularity
popular_df

In [ ]:
popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title').shape

In [ ]:
popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M', 'num_ratings','avg_ratings']]

In [ ]:
#final popular_df
popular_df

## Collaborative Filtering based Recommender System

In [ ]:
x = ratings_with_books.groupby('User-ID').count()['Book-Rating'] > 200
seasoned_readers = x[x].index

In [ ]:
filtered_rating = ratings_with_books[ratings_with_books['User-ID'].isin(seasoned_readers)]

In [ ]:
filtered_rating

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index

In [ ]:
famous_books

In [ ]:
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
final_rating.drop_duplicates()

In [ ]:
pivot_table = final_rating.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [ ]:
pivot_table.fillna(0,inplace=True) 

In [ ]:
pivot_table

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pivot_table)

In [ ]:
similarity_scores

In [ ]:
similarity_scores.shape

In [ ]:
def recommend(book_name):
    #index fetch
    index = np.where(pivot_table.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:11]

    for i in similar_items:
        print(pivot_table.index[i[0]])

In [163]:
#recommends 10 similar books to the given name
recommend('To Kill a Mockingbird')

The Catcher in the Rye
Five Quarters of the Orange
Drowning Ruth
The Bean Trees
The Color of Water: A Black Man's Tribute to His White Mother
Rebecca
Echoes
The Secret Life of Bees
While I Was Gone
Lord of the Flies
